# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 

According to the Naive Bayes assumption, the features are considered independent of each other, so the likelihood $P(\mathbf{x} \mid C)$ can be expressed as the product of the individual likelihoods:

$$
P(\mathbf{x} \mid C) = P(x_1 \mid C) \cdot P(x_2 \mid C) \cdot \ldots \cdot P(x_d \mid C) = \prod_{i=1}^{d} P(x_i \mid C)
$$

Therefore, the posterior probability can be calculated as follows:

$$
P(C \mid \mathbf{x}) = \frac{P(C) \cdot \prod_{i=1}^{d} P(x_i \mid C)}{P(\mathbf{x})}
$$


### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다. 

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다. 

In [5]:
# 사전 확률 구하는 코드를 작성해주세요.
prior_probabilities = df['sentiment'].value_counts(normalize=True)
print("사전 확률:\n", prior_probabilities)

사전 확률:
 sentiment
positive    0.4
negative    0.3
neutral     0.3
Name: proportion, dtype: float64


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [7]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [8]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [9]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [10]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text.split()

df['tokens'] = df['review'].apply(tokenize)

token_by_sentiment = {
    sentiment: sum(df[df['sentiment'] == sentiment]['tokens'], [])
    for sentiment in df['sentiment'].unique()
}

conditional_probabilities = {
    sentiment: {token: (tokens.count(token) + 1) / (len(tokens) + len(set(token_by_sentiment[sentiment])))
                for token in set(tokens)}
    for sentiment, tokens in token_by_sentiment.items()
}

print("조건부 확률:\n", conditional_probabilities)

조건부 확률:
 {'positive': {'very': 0.044444444444444446, 'fast': 0.044444444444444446, 'who': 0.044444444444444446, 'expectations': 0.044444444444444446, 'recommend': 0.044444444444444446, 'exceeded': 0.044444444444444446, 'highly': 0.044444444444444446, 'wonderful': 0.06666666666666667, 'absolutely': 0.044444444444444446, 'with': 0.06666666666666667, 'shipping': 0.044444444444444446, 'can': 0.044444444444444446, 'design': 0.044444444444444446, 'help': 0.044444444444444446, 'satisfied': 0.044444444444444446, 'love': 0.08888888888888889, 'but': 0.044444444444444446, 'quality': 0.044444444444444446, 'great': 0.1111111111111111}, 'negative': {'one': 0.043478260869565216, 'made': 0.043478260869565216, 'brand': 0.043478260869565216, 'purchase': 0.043478260869565216, 'after': 0.043478260869565216, 'have': 0.043478260869565216, 'experience': 0.043478260869565216, 'broke': 0.043478260869565216, 'useless': 0.043478260869565216, 'never': 0.043478260869565216, 'enough': 0.043478260869565216, 'buy': 0

독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [11]:
target_review1 = "love great awesome"
target_review2 = "terrible experience bad"

def calculate_likelihood(review, sentiment):
    tokens = tokenize(review)
    likelihood = [conditional_probabilities[sentiment].get(token, 1 / (len(token_by_sentiment[sentiment]) + len(set(tokens))))
                  for token in tokens]
    return likelihood

print("Likelihoods for target_review1:")
for sentiment in df['sentiment'].unique():
    print(f"Likelihoods for '{sentiment}':", calculate_likelihood(target_review1, sentiment))

print("\nLikelihoods for target_review2:")
for sentiment in df['sentiment'].unique():
    print(f"Likelihoods for '{sentiment}':", calculate_likelihood(target_review2, sentiment))


Likelihoods for target_review1:
Likelihoods for 'positive': [0.08888888888888889, 0.1111111111111111, 0.034482758620689655]
Likelihoods for 'negative': [0.037037037037037035, 0.037037037037037035, 0.037037037037037035]
Likelihoods for 'neutral': [0.037037037037037035, 0.037037037037037035, 0.037037037037037035]

Likelihoods for target_review2:
Likelihoods for 'positive': [0.034482758620689655, 0.034482758620689655, 0.034482758620689655]
Likelihoods for 'negative': [0.06521739130434782, 0.043478260869565216, 0.037037037037037035]
Likelihoods for 'neutral': [0.04878048780487805, 0.037037037037037035, 0.037037037037037035]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [12]:
import numpy as np

def calculate_posterior(review, sentiment):
    prior = prior_probabilities[sentiment]
    likelihood = np.prod(calculate_likelihood(review, sentiment))
    evidence = sum([np.prod(calculate_likelihood(review, s)) * prior_probabilities[s] for s in df['sentiment'].unique()])
    return (likelihood * prior) / evidence if evidence != 0 else 0

# 첫번째 문장
print("P(positive|target_review1):", calculate_posterior(target_review1, 'positive'))
print("P(negative|target_review1):", calculate_posterior(target_review1, 'negative'))
print("P(neutral|target_review1):", calculate_posterior(target_review1, 'neutral'))

# 두번째 문장
print("P(positive|target_review2):", calculate_posterior(target_review2, 'positive'))
print("P(negative|target_review2):", calculate_posterior(target_review2, 'negative'))
print("P(neutral|target_review2):", calculate_posterior(target_review2, 'neutral'))


P(positive|target_review1): 0.8171500630517023
P(negative|target_review1): 0.0914249684741488
P(neutral|target_review1): 0.0914249684741488
P(positive|target_review2): 0.24125583538195833
P(negative|target_review2): 0.4634521557286217
P(neutral|target_review2): 0.29529200888942


A2-1.   
Target review1의 분류 결과: positive   
Target review2의 분류 결과: negative

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 특정 단어가 훈련 데이터의 어떤 클래스에서도 등장하지 않으면 그 단어에 대한 조건부 확률이 0이 되는 문제가 발생함. 이로 인해 해당 클래스에 대한 전체 확률 계산이 왜곡될 수 있음. 이를 해결하기 위해 Laplace smoothing을 적용하면 모든 단어에 작은 확률을 부여하여 해당 문제를 방지할 수 있음.